In [173]:
from sklearn.pipeline import FeatureUnion

In [181]:
from scipy import sparse

In [2]:
import pandas as pd
import numpy as np
import joblib

In [3]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
from scipy import sparse
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [145]:
import lightgbm as lgb

In [188]:
X_train = sparse.load_npz('../sparse/train_matrix.npz')
y_train = pd.read_csv('../sparse/train_labels.csv', names=['label'])

In [184]:
X_test = sparse.load_npz('../sparse/test_matrix.npz')

In [10]:
folds_idx = joblib.load('folds.pkl')

train_df = pd.read_csv('../sparse/train_counter.csv', names=['counter'])

train_df_cat = pd.read_csv('../sparse/train_cat.csv', names=['catergory'])

train_df_diff = pd.read_csv('../sparse/train_day.csv', names=['day_diff'])

df = pd.read_csv('../mlboot_dataset/mlboot_data.tsv.gz',
                 delimiter='\t', 
                 compression='gzip', 
                 header=None,
                 names=['cuid', 'cat_feature', 'counter1', 'counter2', 'counter3', 'dt_diff'])

## Запилим еще числовые фичи и наверное отдельную модель попробуем под это дело посчитать:

 * Самое частую категорию
 * минмакс,среднюю разницу между каждым значением day_diff относительного одного cuid
 * кол-во токенов
 * кол-во токенов суммарное, среднее по дням

In [101]:
features_df = pd.DataFrame()

In [112]:
train_cuid = pd.read_csv('../sparse/train_cuid.csv', names=['cuid'])

In [114]:
mean_cuid_day_count = df.groupby(['cuid', 'dt_diff'], sort=False).count().groupby(['cuid'], sort=False).mean()

In [132]:
mean_cuid_day_count = mean_cuid_day_count.reset_index()

In [65]:
def minmax_days(numbers: str) -> int:
    series = [int(x) for x in numbers.split(' ')]
    return max(series) - min(series)

from typing import Tuple

def min_between(numbers: str) -> int:
    series = [int(x) for x in numbers.split(' ')]
    diffs = np.diff(series)
    try:
        return min(diffs)
    except:
        return 0

def max_between(numbers: str) -> int:
    series = [int(x) for x in numbers.split(' ')]
    diffs = np.diff(series)
    try:
        return max(diffs)
    except:
        return 0
    
def mean_between(numbers: str) -> int:
    series = [int(x) for x in numbers.split(' ')]
    diffs = np.diff(series)
    mean_diff = np.mean(diffs)
    
    if np.isnan(mean_diff):
        return 0
    return mean_diff


In [133]:
features_df['mean_cuid_day_count'] = mean_cuid_day_count['counter1']

In [71]:
train_df_diff['minmax_days'] = train_df_diff.day_diff.apply(lambda x: minmax_days(x))

In [28]:
train_df_cat['mode_category'] = train_df_cat.catergory.apply(lambda x: max(set(x.split(' ')), key=x.split(' ').count))

In [66]:
train_df_diff['min_day_diff'] = train_df_diff.day_diff.apply(lambda x: min_between(x))
train_df_diff['max_day_diff'] = train_df_diff.day_diff.apply(lambda x: max_between(x))
train_df_diff['mean_day_diff'] = train_df_diff.day_diff.apply(lambda x: mean_between(x))


/home/chislov/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [102]:
features_df['minmax_days'] = train_df_diff['minmax_days']

In [ ]:
features_df['min_day_diff'] = train_df_diff['min_day_diff']
features_df['max_day_diff'] = train_df_diff['max_day_diff']
features_df['mean_day_diff'] = train_df_diff['mean_day_diff']
features_df['mode_category'] = train_df_cat['mode_category']

In [167]:
features_df.head()

,minmax_days,min_day_diff,max_day_diff,mean_day_diff,mode_category
0,16,1,5,2.666667,5
1,1,1,1,1.000000,5
2,0,0,0,0.000000,5
3,50,0,41,6.250000,1
4,57,1,10,4.750000,5


In [135]:
features_df.to_csv('feat.csv', index=False)

### Test features

In [156]:
test_df_diff = pd.read_csv('../sparse/test_counter.csv', names=['day_diff'])

test_df_counter = pd.read_csv('../sparse/test_cat.csv', names=['counter'])

test_df_category = pd.read_csv('../sparse/test_day.csv', names=['category'])

In [155]:
train_df_diff.head()

,day_diff
0,2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 ...
1,2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 ...
2,2 3 2 3 2 3 2 3 2 2 2 3 3 2 3 2 3 3 2 2 3 3 3 ...
3,3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 ...
4,5 3 5 5 5 5 5 3 3 5 5 3 5 3 3 5 3 5 3 5 3 5 5 ...


In [163]:
train_df_diff['minmax_days'] = train_df_diff.day_diff.apply(lambda x: minmax_days(x))
train_df_category['mode_category'] = train_df_category.category.apply(lambda x: max(set(x.split(' ')), key=x.split(' ').count))
train_df_diff['min_day_diff'] = train_df_diff.day_diff.apply(lambda x: min_between(x))
train_df_diff['max_day_diff'] = train_df_diff.day_diff.apply(lambda x: max_between(x))
train_df_diff['mean_day_diff'] = train_df_diff.day_diff.apply(lambda x: mean_between(x))
test_features_df = pd.DataFrame()
test_features_df['minmax_days'] = train_df_diff['minmax_days']

test_features_df['min_day_diff'] = train_df_diff['min_day_diff']
test_features_df['max_day_diff'] = train_df_diff['max_day_diff']
test_features_df['mean_day_diff'] = train_df_diff['mean_day_diff']
test_features_df['mode_category'] = train_df_category['mode_category']

In [150]:
features_df['mode_category'] = features_df['mode_category'].astype('int')

In [164]:
test_features_df.head()

,minmax_days,min_day_diff,max_day_diff,mean_day_diff,mode_category
0,52,1,5,1.333333,2
1,59,1,3,1.255319,2
2,45,0,10,1.730769,3
3,59,1,4,1.180000,3
4,59,0,2,0.641304,5


In [165]:
test_features_df.to_csv('test_feat.csv', index=False)

## Соберем все в один  датасет

In [138]:
X_train = features_df
y_train = pd.read_csv('../sparse/train_labels.csv', names=['label'])


In [171]:
features_df.head()

,minmax_days,min_day_diff,max_day_diff,mean_day_diff,mode_category
0,16,1,5,2.666667,5
1,1,1,1,1.000000,5
2,0,0,0,0.000000,5
3,50,0,41,6.250000,1
4,57,1,10,4.750000,5


In [177]:
num_matrix = features_df.as_matrix()

/home/chislov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [189]:
concatenated = sparse.hstack([X_train, num_matrix])

In [191]:
test_features_df['mode_category'] = test_features_df['mode_category'].astype('int')

In [192]:
test_concatenated = sparse.hstack([X_test, test_features_df.as_matrix()])

/home/chislov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [193]:
sparse.save_npz('concat_features_train.npz', concatenated)

In [ ]:
sparse.save_npz('concat_features_test.npz', test_concatenated)